In [ ]:
import os
import numpy as np
from PIL import Image, ImageOps, ImageEnhance
import random
from tqdm import tqdm
from google.colab import drive

# load Google Drive
drive.mount('/content/drive')

# AugMix Augmented Operations
def augmentations():
    return [
        (lambda x: ImageOps.autocontrast(x)),  # Auto contrast
        (lambda x: ImageOps.equalize(x)),  # equalization
        (lambda x: ImageOps.solarize(x, threshold=128)),  # solarization
        (lambda x: x.rotate(random.choice([15, 30, 45]))),  # random spin
        (lambda x: ImageEnhance.Color(x).enhance(random.uniform(1.2, 2.0))),  # color enhancement
        (lambda x: ImageEnhance.Brightness(x).enhance(random.uniform(1.2, 2.0))),  # Brightness enhancement
        (lambda x: ImageEnhance.Contrast(x).enhance(random.uniform(1.2, 2.0))),  # Contrast enhancement
        (lambda x: ImageEnhance.Sharpness(x).enhance(random.uniform(1.2, 2.0))),  # sharpen
    ]

# Randomly select and apply an augmentation operation
def apply_op(image, op):
    return op(image)

# AugMix Core Functions
def augment_and_mix(image, severity=3, width=3, depth=-1, alpha=1.0):
    """Perform AugMix augmentations and compute mixture."""
    ws = np.float32(np.random.dirichlet([alpha] * width))  # The weight of each reinforcement chain
    m = np.float32(np.random.beta(alpha, alpha))  # The weight of the original image and the enhanced image

    mix = np.zeros_like(np.asarray(image), dtype=np.float32)
    for i in range(width):
        image_aug = image.copy()
        d = depth if depth > 0 else np.random.randint(1, 4)  # random depth
        for _ in range(d):
            op = random.choice(augmentations())  # Randomly select augmentation operations
            image_aug = apply_op(image_aug, op)

        mix += ws[i] * np.asarray(image_aug, dtype=np.float32) / 255.0

    mixed = (1 - m) * np.asarray(image, dtype=np.float32) / 255.0 + m * mix
    return np.clip(mixed, 0, 1)  # Ensure pixel values ​​are in the range [0, 1]

# AugMix Visual Operation
def augment_visualize(image_path, save_dir):
    """
    AugMix augments a single image and saves it to the specified directory
    """
    try:
        # Open picture
        image = Image.open(image_path).convert("RGB")
        
        # AugMix augmented
        augmented_image = augment_and_mix(image)
        
        # Perform AugMix operation on the augmented image and the original image again
        final_image = augment_and_mix(Image.fromarray((augmented_image * 255).astype(np.uint8)))
        
        # Save the final AugMix augmented image
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, "augmix_" + os.path.basename(image_path))
        final_image = Image.fromarray((final_image * 255).astype(np.uint8))
        final_image.save(save_path)
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

# Traverse the folder and process the images
def process_all_images(input_dir):
    """
    Traverse the four subfolders, perform AugMix on all images and save them
    """
    subfolders = ["cataract", "diabetic_retinopathy", "glaucoma", "normal"]
    for subfolder in subfolders:
        folder_path = os.path.join(input_dir, subfolder)
        augmix_folder = os.path.join(folder_path, "AugMix")
        for root, _, files in os.walk(folder_path):
            for file in tqdm(files, desc=f"Processing {subfolder}"):
                if file.lower().endswith((".jpg", ".jpeg", ".png")):  # Process image files only
                    image_path = os.path.join(root, file)
                    augment_visualize(image_path, augmix_folder)

# Directory path
input_dir = "/content/drive/My Drive/contenteye_diseases/Training"

process_all_images(input_dir)
